## Lógica Computacional: 25/26
---
## TP1 - Ex1

$Grupo$ $05$ 

*   Vasco Ferreira Leite (A108399)
*   Gustavo da Silva Faria (A108575)
*   Afonso Henrique Cerqueira Leal (A108472)
---

### Descrição do problema

Este problema usa optimização MIP (“Mixed Integer Programming” (OrTools) e representação por  Grafos ( NetworkX)).

1. Para um distribuidor de encomendas o seu território está organizados em pontos (“nodes”) de fornecimento (“sources”), pontos de passagem  e pontos de entrega (“sinks”) ligados por vias de comunicação (“edges”) bidirecionais cada uma das quais associada uma capacidade em termos do número de veículos de transporte que suporta.
2. Os items distribuidos estão organizados em “pacotes” de três tipos “standard” : uma unidade, duas unidades e cinco unidades. Os pacotes são transportados em veículos todos com a capacidade de 10 unidades. Cada ponto de entrega tem um limite no número total de unidades que tem em “stock” e um limite no número de veículos que dispõe.
3. Cada encomenda é definida por o identificador do ponto de entrega e pelo número de pacotes, de cada um dos tipos, que devem ser entregues nesse ponto.
4. O objetivo do problema é decidir, a partir de uma encomenda e com um mínimo no número de veículos:

        Em cada ponto de entrega,  se estará envolvido no fornecimento de unidades que essa encomenda requer sem violar os limites do seu “stock”.

        Em cada ponto de entrega,   como empacotar as unidades disponíveis, de acordo com a encomenda”,  e como as distribuir por veículos,

        Em cada veículo, qual o percurso a seguir até  ao ponto de entrega; para cada via ao longo de cada percurso, o total de veículos não pode exceder a capacidade dessa via.

### **Resolução do problema**

##### **Variáveis**

meter aqui as merdas referentes a variaveis 

##### **Importação de Bibliotecas**

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

*   `NetworkX`: Para modelar e manipular o grafo da rede de distribuição
*   `Matplotlib`: Para visualização gráfica da rede
*   `OR-Tools`: Para resolver o problema de otimização MIP

#### **Funções**

*   ##### Função criar_rede

In [3]:
def criar_rede():
    """Cria a rede de distribuição"""
    G = nx.DiGraph()
    
    # Sources
    G.add_node('S1', type='source', color='green', stock=40, veiculos=8)
    G.add_node('S2', type='source', color='green', stock=45, veiculos=9)
    
    # Pontos de passagem
    G.add_node('H1', type='passagem', color='lightblue')
    G.add_node('H2', type='passagem', color='lightblue')
    G.add_node('P1', type='passagem', color='skyblue')
    G.add_node('P2', type='passagem', color='skyblue')
   
    # Destinos
    G.add_node('D1', type='sink', color='red')
    G.add_node('D2', type='sink', color='red')
    G.add_node('D3', type='sink', color='red')
    G.add_node('D4', type='sink', color='red')
   
    # Vias (origem, destino, capacidade)
    vias = [
        ('S1', 'H1', 12), ('S1', 'H2', 9),
        ('S2', 'H1', 10), ('S2', 'P2', 8),
        ('H1', 'H2', 9), ('H1', 'P1', 8), ('H1', 'P2', 9), ('H1', 'D1', 5), ('H1', 'D3', 6), 
        ('H2', 'P1', 6), ('H2', 'D2', 4),
        ('P1', 'P2', 6), ('P1', 'D1', 7), ('P1', 'D2', 6),
        ('P2', 'D2', 7), ('P2', 'D3', 8), ('P2', 'D4', 9), 
        ('D1', 'D2', 4), ('D3', 'D4', 5),
    ]
    
    for orig, dest, cap in vias:
        G.add_edge(orig, dest, capacity=cap)
        G.add_edge(dest, orig, capacity=cap)
    
    return G

Esta função é responsável por construir a rede de distribuição.
Para isso, são criados pontos (“nodes”) de fornecimento (“sources”), pontos de passagem  e pontos de entrega (“sinks”) ligados por vias de comunicação (“edges”) . Às sources estão atríbuidas um stock (unidades disponíveis) e um número máximo de veículos. As vias uma capacidade são bidirecionais e está lhes atribuído uma capacidade que representa o número máximo de veículos.

*   ##### Função visualizar_rede

In [4]:
def visualizar_rede(G):
    """Visualiza a rede de distribuição"""
    sources = [n for n in G.nodes if G.nodes[n]['type'] == 'source']
    sinks = [n for n in G.nodes if G.nodes[n]['type'] == 'sink']
    passagem = [n for n in G.nodes if G.nodes[n]['type'] == 'passagem']
        
    pos = {}
    for i, node in enumerate(sources):
        pos[node] = (i * 2, 3)
    
    hubs = [n for n in passagem if n.startswith('H')]
    intermedios = [n for n in passagem if n.startswith('P')]
    
    for i, node in enumerate(hubs):
        pos[node] = (i * 2.5 + 0.5, 2)
    for i, node in enumerate(intermedios):
        pos[node] = (i * 2 + 1, 1)
    for i, node in enumerate(sinks):
        pos[node] = (i * 1.5 + 0.5, 0)
    
    plt.figure(figsize=(14, 10))
  
    node_colors = [G.nodes[n]['color'] for n in G.nodes]
    
    nx.draw_networkx_edges(G, pos, width=1, alpha=1, 
                          edge_color='black', arrows=True, arrowsize=15)
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                          node_size=1200, edgecolors='black')
    nx.draw_networkx_labels(G, pos, font_size=9, font_weight='bold')
    
    edge_labels = {}
    edges_processed = set()
    for u, v in G.edges():
        if (v, u) not in edges_processed:
            edge_labels[(u, v)] = G[u][v]['capacity']
            edges_processed.add((u, v))
    
    nx.draw_networkx_edge_labels(G, pos, edge_labels, alpha=1)
    
    plt.title("Rede de Distribuição", fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
